In [41]:
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, ConstantKernel
import numpy as np
from stockdex import Ticker
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore")

def get_data(range, dataGranularity, stock):
    ticker = Ticker(stock)
    try:
        data = ticker.yahoo_api_price(range=range, dataGranularity=dataGranularity)
        df = pd.DataFrame({
            'timestamp': data['timestamp'],
            'close': data['close'],
        })
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        return df
    except Exception as e:
        print(f"An error occurred while fetching data for {stock}: {e}")
        return None

def best_gpr(X_train, y_train, X_test, y_test, kernels, **gpr_params):
    """
    Trenuje GPR z różnymi kernelami i zwraca model z najlepszym kernelem (najniższe MSE).

    Args:
        X_train: dane treningowe (features)
        y_train: dane treningowe (target)
        X_test: dane testowe (features)
        y_test: dane testowe (target)
        kernels: lista kernelów do przetestowania
        **gpr_params: dodatkowe parametry dla GaussianProcessRegressor

    Returns:
        best_model: wytrenowany GPR z najlepszym kernelem
        best_score: MSE najlepszego modelu
    """
    best_score = float('inf')
    best_model = None

    for kernel in kernels:
        gpr = GaussianProcessRegressor(kernel=kernel, **gpr_params)
        gpr.fit(X_train, y_train)
        y_pred = gpr.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        print(f"Kernel: {gpr.kernel_}, MSE: {mse:.4f}")

        if mse < best_score:
            best_score = mse
            best_model = gpr

    print(f"\n✅ Najlepszy kernel: {best_model.kernel_}, MSE: {best_score:.4f}")
    return best_model, best_score


data = get_data('1y', '1d', 'TSLA')
# X i y
X = np.arange(len(data)).reshape(-1, 1)
y = data['close'].values

# Podział: 90% trening, 10% test
split_idx = int(0.95 * len(data))
X_train, y_train = X[:split_idx], y[:split_idx]
X_test, y_test = X[split_idx:], y[split_idx:]
dates_train = data.index[:split_idx]
dates_test = data.index[split_idx:]

# Model
kernel_list = [
    RBF(length_scale=5.0) + WhiteKernel(noise_level=0.5),
    RBF(length_scale=10.0) + WhiteKernel(noise_level=1.0),
    Matern(length_scale=10.0, nu=1.5) + WhiteKernel(noise_level=1.0),
    ConstantKernel(1.0) * RBF(length_scale=20.0) + WhiteKernel(noise_level=0.5)
]

# Wywołanie funkcji
gpr, best_mse = best_gpr(
    X_train, y_train, X_test, y_test,
    kernels=kernel_list,
    normalize_y=True,
    n_restarts_optimizer=10
)

# Predykcja na wszystkich punktach (trening + test)
X_all = np.vstack([X_train, X_test])
y_pred, y_std = gpr.predict(X_all, return_std=True)

plt.figure(figsize=(12, 6))

# Ceny rzeczywiste (treningowe)
plt.plot(dates_train, y_train, 'r.', markersize=4, label='Rzeczywiste (trening)')

# Ceny rzeczywiste (testowe)
plt.plot(dates_test, y_test, 'g.', markersize=6, label='Rzeczywiste (test)')

# Predykcja
plt.plot(data.index, y_pred, 'b-', label='Predykcja GPR')

# Przedział ufności
plt.fill_between(data.index,
                 y_pred - 1.645 * y_std, y_pred + 1.645 * y_std,
                 alpha=0.2, color='blue', label='90% przedział ufności')

# Linia podziału
plt.axvline(dates_test[0], color='gray', linestyle='--', label='Początek testu')

plt.title('Gaussian Process Regressor - GLD (90% trening + 10% test)')
plt.xlabel('Data')
plt.ylabel('Cena zamknięcia')
plt.legend()
plt.show()

An error occurred while fetching data for TSLA: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/TSLA?range=1y&interval=1d (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x14fe9d9d0>: Failed to resolve 'query2.finance.yahoo.com' ([Errno 8] nodename nor servname provided, or not known)"))


TypeError: object of type 'NoneType' has no len()